In [1]:
import matplotlib.pyplot as plt 
import numpy as np
import random
import keras
%matplotlib inline

Using TensorFlow backend.


In [2]:
# Load training text file
Data_train = np.loadtxt('training_data.txt', skiprows = 1)
X_train = Data_train[:, 1:]
y_train = Data_train[:, 0]
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

X_train shape: (20000, 1000)
y_train shape: (20000,)


In [45]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import Conv2D, MaxPooling2D, Flatten, BatchNormalization
from keras import regularizers

# Build sequential model
model = Sequential()

# Hidden layers
model.add(Dense(500, activation="tanh", input_shape=(1000,)))
model.add(Dropout(0.4))
model.add(Dense(100, activation="tanh"))
model.add(Dropout(0.4))
# model.add(Dense(50, activation="tanh"))
# model.add(Dropout(0.4))

# Output layer
model.add(Dense(1, activation="sigmoid"))
# Print a summary
model.summary()
# Compile
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_59 (Dense)             (None, 500)               500500    
_________________________________________________________________
dropout_44 (Dropout)         (None, 500)               0         
_________________________________________________________________
dense_60 (Dense)             (None, 100)               50100     
_________________________________________________________________
dropout_45 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_61 (Dense)             (None, 1)                 101       
Total params: 550,701
Trainable params: 550,701
Non-trainable params: 0
_________________________________________________________________


In [46]:
fit = model.fit(X_train, y_train, batch_size=128, epochs=5, validation_split=0.2, shuffle=True, verbose=1)

Train on 16000 samples, validate on 4000 samples
Epoch 1/5
16000/16000 [==============================] - 5s 296us/step - loss: 0.4209 - acc: 0.8075 - val_loss: 0.3553 - val_acc: 0.8472
Epoch 2/5
16000/16000 [==============================] - 4s 267us/step - loss: 0.3458 - acc: 0.8486 - val_loss: 0.3518 - val_acc: 0.8508
Epoch 3/5
16000/16000 [==============================] - 4s 247us/step - loss: 0.3310 - acc: 0.8557 - val_loss: 0.3502 - val_acc: 0.8538
Epoch 4/5
16000/16000 [==============================] - 4s 253us/step - loss: 0.3216 - acc: 0.8609 - val_loss: 0.3513 - val_acc: 0.8525
Epoch 5/5
16000/16000 [==============================] - 4s 237us/step - loss: 0.3115 - acc: 0.8667 - val_loss: 0.3499 - val_acc: 0.8530


In [47]:
## Printing the accuracy of our model, according to the loss function specified in model.compile above
score = model.evaluate(X_train, y_train, verbose=1)
print('Training score:', score[0])
print('Training accuracy:', score[1])

20000/20000 [==============================] - 3s 141us/step
Training score: 0.291117485988
Training accuracy: 0.8752


In [7]:
# Make predictions
X_test = np.loadtxt('test_data.txt', skiprows = 1)

pred = model.predict(X_test)
super_threshold_indices = pred >= 0.5
pred.fill(0)
pred[super_threshold_indices] = 1

submission = [[i+1, int(pred[i])] for i in range(len(pred))]
submission.insert(0, ['Id','Prediction'])
with open('submission_neural_net.csv', 'w') as f:
    for line in submission:
        f.write(','.join(map(str, line)) + '\n')

In [8]:
print(pred[0:5])

[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]]
